**Etapa 2.1 e 2.2: EDA e Limpeza Focada**

In [2]:
import pandas as pd
import os
import plotly.express as px

# --- Configurações ---
FILE_NAME = 'sigsifrelatorioabatesporanouf.csv'
FILE_PATH = os.path.join('../data/raw/', FILE_NAME)
CHUNKSIZE = 200_000 # Podemos usar um chunksize maior pois o arquivo não é tão grande

# --- Etapa de Pré-processamento e Carga ---
print(f"Iniciando o processamento do arquivo: {FILE_NAME}")

processed_chunks = []
try:
    # Vamos ler o arquivo em pedaços para garantir a eficiência
    iterator = pd.read_csv(FILE_PATH, sep=';', encoding='latin1', chunksize=CHUNKSIZE, low_memory=False)

    for chunk in iterator:
        # 1. Renomear colunas para facilitar o uso
        chunk.rename(columns={
            'MES_ANO': 'data',
            'UF_PROCEDENCIA': 'uf',
            'CATEGORIA': 'especie',
            'QTD': 'quantidade_cabecas'
        }, inplace=True)

        # 2. Converter a coluna 'data' para o formato datetime
        # O formato é 'MM/YYYY', então ajudamos o pandas a entender
        chunk['data'] = pd.to_datetime(chunk['data'], format='%m/%Y', errors='coerce')

        # 3. Remover linhas onde a data não pôde ser convertida
        chunk.dropna(subset=['data'], inplace=True)

        # 4. Otimizar tipos de dados para reduzir uso de memória
        chunk['quantidade_cabecas'] = pd.to_numeric(chunk['quantidade_cabecas'], downcast='integer')
        chunk['uf'] = chunk['uf'].astype('category')
        chunk['especie'] = chunk['especie'].astype('category')

        processed_chunks.append(chunk)

except Exception as e:
    print(f"Ocorreu um erro durante o processamento: {e}")

# Concatena todos os pedaços processados em um único DataFrame
df_abate = pd.concat(processed_chunks)

print("\nProcessamento concluído. Verificando a estrutura final do DataFrame:")
df_abate.info()

print("\nVerificando as espécies presentes no dataset:")
print(df_abate['especie'].unique())

print("\n--- Amostra dos Dados Limpos e Estruturados ---")
display(df_abate.head())


# --- Etapa de Análise Exploratória (EDA) ---

# 1. Agregar dados por ano para a visualização macro
# Extraímos o ano da coluna 'data'
# Linha corrigida para silenciar o warning e adotar a prática futura
df_anual = df_abate.groupby(
    [df_abate['data'].dt.year, 'especie'],
    observed=True  # Adicionamos este parâmetro
)['quantidade_cabecas'].sum().reset_index()

df_anual.rename(columns={'data': 'ano'}, inplace=True)

print("\n--- Dados Agregados por Ano ---")
display(df_anual.head())

# 2. Criar o gráfico de linhas interativo
print("\n--- Visualização: Série Temporal de Abates por Espécie ---")
fig = px.line(
    df_anual,
    x='ano',
    y='quantidade_cabecas',
    color='especie',
    title='Evolução Anual do Abate de Animais no Brasil (SIF)',
    labels={
        'ano': 'Ano',
        'quantidade_cabecas': 'Quantidade de Cabeças Abatidas',
        'especie': 'Espécie'
    },
    markers=True
)

fig.show()

Iniciando o processamento do arquivo: sigsifrelatorioabatesporanouf.csv

Processamento concluído. Verificando a estrutura final do DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59563 entries, 0 to 59562
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   data                59563 non-null  datetime64[ns]
 1   uf                  59563 non-null  category      
 2   especie             59563 non-null  category      
 3   quantidade_cabecas  59563 non-null  int32         
dtypes: category(2), datetime64[ns](1), int32(1)
memory usage: 818.5 KB

Verificando as espécies presentes no dataset:
['Bovino', 'Suino', 'Equino', 'Novilho Precoce', 'Leitao', ..., 'Cutia', 'Ostra', 'Uma ou mais espÃ©cies', 'Aves nÃ£o destinadas Ã  produÃ§Ã£o de carne o..., 'Pirapitinga']
Length: 92
Categories (92, object): ['Asinino', 'Aves', 'Aves nÃ£o destinadas Ã  produÃ§Ã£o de carne o..., 'Avestruz', ...

,data,uf,especie,quantidade_cabecas
0,2002-01-01,DF,Bovino,14
1,2003-01-01,BA,Suino,2531
2,2003-01-01,MG,Equino,1469
3,2003-01-01,MT,Novilho Precoce,647
4,2003-01-01,PR,Suino,157336



--- Dados Agregados por Ano ---


,ano,especie,quantidade_cabecas
0,1995,Bovino,245
1,2000,Bovino,37900
2,2000,Suino,14085
3,2001,Bovino,41978
4,2001,Frango,13405752



--- Visualização: Série Temporal de Abates por Espécie ---
